# Bike Sharing Demand
by Sean Park  
2020.04.24
* Goal: RMSLE - under 0.400

# Data Fields
* datetime - hourly date + timestamp  
* season
    * 1 = spring, 
    * 2 = summer, 
    * 3 = fall, 
    * 4 = winter 
* holiday - whether the day is considered a holiday
* workingday - whether the day is neither a weekend nor holiday
* weather 
    * 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    * 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    * 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    * 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
* temp - temperature in Celsius
* atemp - "feels like" temperature in Celsius
* humidity - relative humidity
* windspeed - wind speed
* casual - number of non-registered user rentals initiated
* registered - number of registered user rentals initiated
* count - number of total rentals

# 데이터 불러오고 형태 파악

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 폰트 크기
plt.rc('font', size=13)
plt.rc('font', family='NanumGothic')

In [ ]:
df_train = pd.read_csv('../input/bike-sharing-demand/train.csv')
df_test = pd.read_csv('../input/bike-sharing-demand/test.csv')

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
print('Null in train : {}\nNull in test: {}'.format(df_train.isnull().sum().sum(), df_test.isnull().sum().sum()))

In [ ]:
print('df_train.shape: {}\ndf_test.shape: {}'.format(df_train.shape, df_test.shape))

* 테스트 세트에는 registeed, casual, count 컬럼이 없음

In [ ]:
df_train.head()

In [ ]:
df_train.tail()

In [ ]:
df_test.head()

In [ ]:
df_test.tail()

In [ ]:
df_train.datetime.value_counts().head(10)

* datetime 컬럼의 형식을 object에서 날짜 형식으로 변경하자.
    * year, month, day, hour, day of week 로 구분하도록 하자.
    * minute와 seconds는 알아도 크게 유의미하지 않다고 판단되고, 어차피 값이 모두 0이다.

In [ ]:
# pandas 
for df in [df_train, df_test]: df['datetime'] = pd.to_datetime(df['datetime'])

In [ ]:
for df in [df_train, df_test]:
    df['year'] = df.datetime.dt.year
    df['month'] = df.datetime.dt.month
    df['day'] = df.datetime.dt.day
    df['hour'] = df.datetime.dt.hour
    df['dow'] = df.datetime.dt.dayofweek
df_train.head()

In [ ]:
numeric_col = ['temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count']
df_train[numeric_col].describe()

* 각 컬럼들의 평균과 분포를 확인할 수 있다.
* casual 이용자보다 registered 이용자가 더 많다.

# EDA
* 데이터를 이해하기 위해 탐색하고, 시각화해보자.

## 0. 상관관계 히트맵
* 먼저, 각 피처들 간 상관관계를 시각화해보자.

In [ ]:
df_train.head()

In [ ]:
col_numeric = ['temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count']
plt.figure(figsize=(10, 10))
sns.heatmap(df_train[col_numeric].corr(), annot=True, fmt='.2f')
plt.title('Correlation')

* 온도, 체감온도와 이용량이 양의 상관관계를 보인다.
* 습도와 이용량이 양의 상관관계에 있다.
    * 습도가 높을수록 불쾌지수가 높아져서 그런가?
* 풍속은 상관관계가 0.1미만으로 약하다.

## 1. Season
*Description  
1 = spring, 2 = summer, 3 = fall, 4 = winter*
* 계절에 따른 이용 변화를 파악해보자.
* 추운 날보다는 따뜻한 날에 이용이 높을 것으로 예상되는데, 이를 중점으로 살펴보자.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(15, 5))
color = ['#1E82E6', '#E6C112', '#E66C29']
sns.barplot('season', 'count', data=df_train, ax=ax[0])
ax[0].set_title('AVG of user by season (total)')

sns.barplot('season', 'registered', data=df_train, color=color[0], ax=ax[1], label='registered')
sns.barplot('season', 'casual', data=df_train, color=color[1], ax=ax[1], label='casual')
ax[1].legend(loc=2)
ax[1].set_title('AVG of user by season (user seperated)')

# y축 고정
for ax in [ax[0], ax[1]]:
    ax.set_ylim(0,250)

* 봄의 이용자가 가장 적었고, 가을의 이용자가 가장 많았다.
* registered, casual 이용자 모두 계절에 따른 변화 추이가 비슷하다.
    * 한 가지 다른점은, casual 이용자의 겨울 감소폭이 상대적으로 높다.

## 2. Holiday
* 휴일의 이용 상황을 파악해보자.

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(20, 5))
sns.countplot('holiday', data=df_train, ax=ax[0])
ax[0].set_title('Count of Holiday')

sns.barplot('holiday', 'count', data=df_train, ax=ax[1])
ax[1].set_title('AVG of user by holiday')

sns.boxplot('holiday', 'count', data=df_train, ax=ax[2])
ax[2].set_title('Boxplot of user by holiday')
plt.show();
print(df_train.holiday.value_counts())

* 샘플 차이가 많이 나기때문에 평균 뿐만 아니라, 사분위값도 함께 확인
* 평균, 중앙값에 큰 차이가 없는 것으로 보인다.
* 그러면 유저별로 나눠서 살펴보자.

In [ ]:
f, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=(12, 12))
sns.barplot('holiday', 'registered', data=df_train, ax=ax0)
ax0.set_title('AVG of registered user by holiday')

sns.barplot('holiday', 'casual', data=df_train, ax=ax1)
ax1.set_title('AVG of casual user by holiday')

for ax in [ax0, ax1]:
    ax.set_ylim(0, 160)
    
sns.boxplot('holiday', 'registered', data=df_train, ax=ax2)
ax2.set_title('Boxplot of registered user by holiday')

sns.boxplot('holiday', 'casual', data=df_train, ax=ax3)
ax3.set_title('Boxplot of casual user by holiday')
plt.show();

* 분리해서 살펴보니, registered 이용자는 휴일에 이용이 감소하는 반면, casual 유저는 증가한다.
* 하지만 앞에서 언급했듯 두 집단은 샘플 차이가 많이 나기 때문에, 아직 유의미한 차이라고 하기는 어렵다.
    * Q1 ~ Q3 부분의 범위도 많이 겹치는 것을 볼 수 있다.
* 실제 차이가 있는지 검정을 해보자.

In [ ]:
from scipy import stats

for user in ['registered', 'casual']:
    holiday = df_train[df_train['holiday']==0][user]
    n_holiday = df_train[df_train['holiday']==1][user]
    tTest = stats.ttest_ind(holiday, n_holiday, equal_var=False)
    print('{}의 tTest 결과:\nt통계량: {:.2f}, p값: {:.4f}'.format(user, tTest[0], tTest[1]))

* 실제 평균 차이가 있음을 확인할 수 있다.

## 3. Workingday

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 5))
sns.countplot('workingday',data=df_train, ax=ax[0])
ax[0].set_title('Count of workingday')
sns.barplot('workingday', 'count', data=df_train, ax=ax[1])
ax[1].set_title('AVG user by workingday')

* 전체 유저를 비교해보니, 근무일 여부에 따른 이용수 평균은 큰 차이가 없어보인다.

In [ ]:
f, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=(12, 12))
sns.barplot('workingday', 'registered', data=df_train, ax=ax0)
ax0.set_title('AVG of registered user by workingday')

sns.barplot('workingday', 'casual', data=df_train, ax=ax1)
ax1.set_title('AVG of casual user by workingday')

for ax in [ax0, ax1]: ax.set_ylim(0, 180)

sns.boxplot('workingday', 'registered', data=df_train, ax=ax2)
ax2.set_title('Boxplot of registered user by workingday')

sns.boxplot('workingday', 'casual', data=df_train, ax=ax3)
ax3.set_title('Boxplot of casual user by workingday')
plt.show();

* 하지만 유저를 나누어보니, registered 유저는 이용이 증가하고 casual 유저는 감소한다.
    * 휴일과 반대인 경향을 보인다.
* registered 유저는 출퇴근 목적으로 자전거를 이용하고, casual은 여가, 취미 등으로 이용하는 것일까?

## 4. Weather

*Description  
1: Clear, Few clouds, Partly cloudy, Partly cloudy  
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist  
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds  
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog*
        
* 날씨에 따른 이용 추이를 살펴보자.
* 상식적으로는 날씨가 좋을수록*(1에 가까울수록)* 이용이 증가할 것으로 예상된다.

In [ ]:
sns.countplot('weather', data=df_train)
plt.title('Count of weather')
plt.show();
print(df_train.weather.value_counts())

* 날씨가 좋은 날(1)이 가장 많았고, 날이 가장 안좋았던 날(4)은 하루밖에 없었다.
* 4에 대한 샘플은 하나뿐임을 감안하고 볼 것

In [ ]:
plt.figure(figsize=(7, 5))
sns.barplot('weather', 'registered', data=df_train, color=color[0], label='registered')
sns.barplot('weather', 'casual', data=df_train, color=color[1], label='casual')
plt.ylim(0, 175)
plt.title('AVG of user by weather (seperated)')
plt.legend(loc=(1.02, 0.85))

* 예상대로 날씨가 좋은 날에 이용이 높았다.
* 날씨가 가장 안 좋은날(4)에는 이용이 높았으나, 샘플이 하나이므로 큰 의미는 없다.

## 5. Temp, Atemp
* 온도, 체감온도와 이용수와의 관계를 보자.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 5))
for ax, col in zip([ax[0], ax[1]], ['temp', 'atemp']):
    sns.distplot(df_train[col], ax=ax)
    ax.set_title('Histogram of {}'.format(col))

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 5))
for ax, col in zip([ax[0], ax[1]], ['temp', 'atemp']):
    sns.scatterplot(col, 'count', data=df_train, ax=ax)
    ax.set_title('Scatterplot of {}:count (r={:.2f})'.format(col, df_train['count'].corr(df_train[col])))

In [ ]:
f, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=(14, 12))
for ax, col in zip([ax0, ax1], ['temp', 'atemp']):
    sns.scatterplot(col, 'registered', data=df_train, ax=ax)
    ax.set_title('Scatterplot of {}:registered (r={:.2f})'.format(col, df_train['registered'].corr(df_train[col])))

for ax, col in zip([ax2, ax3], ['temp', 'atemp']):
    sns.scatterplot(col, 'casual', data=df_train, ax=ax)
    ax.set_title('Scatterplot of {}:casual (r={:.2f})'.format(col, df_train['casual'].corr(df_train[col])))

* 온도, 체감온도와 이용수가 관계가 양의 상관관계에 있다.
* 특히 casual 이용자의 경우, registered 이용자에 비해 상관계수가 더 높다.

## 6. Humidity
* 습도가 높을수록 불쾌지수가 높아지면서 이용수가 감소할 것으로 예상되는데, 한 번 살펴보자.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 5))
sns.distplot(df_train['humidity'], ax=ax[0])
ax[0].set_title('Histogram of humidity')

sns.scatterplot('humidity', 'count', data=df_train, ax=ax[1])
ax[1].set_title('Scatterplot of humidity:count (r = {:.2f})'.format(df_train['count'].corr(df_train['humidity'])))

In [ ]:
f, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 5))
for ax, user in zip([ax0, ax1], ['registered', 'casual']):
    sns.scatterplot('humidity', user, data=df_train, ax=ax)
    ax.set_title('Scatterplot of humidity:{} (r={:.2f})'.format(user, df_train[user].corr(df_train['humidity'])))

* 이용수와 습도는 서로 음의 상관관계에 있고, casual유저의 상관계수가 registered 유저보다 높다.

## 7. Windspeed
* 풍속과의 관계를 살펴보자.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 5))
sns.distplot(df_train['windspeed'], ax=ax[0])
ax[0].set_title('Histogram of windspeed')

sns.scatterplot('windspeed', 'count', data=df_train, ax=ax[1])
ax[1].set_title('Scatterplot of windspeed:count (r = {:.2f})'.format(df_train['count'].corr(df_train['windspeed'])))

In [ ]:
f, (ax0, ax1) = plt.subplots(1, 2, figsize=(12, 5))
for ax, user in zip([ax0, ax1], ['registered', 'casual']):
    sns.scatterplot('windspeed', user, data=df_train, ax=ax)
    ax.set_title('Scatterplot of windspeed:{} (r={:.2f})'.format(user, df_train[user].corr(df_train['windspeed'])))

* 바람과의 상관관계는 0.09로 낮다.
* 그리고 풍속이 0으로 집계된 날이 많다.

## 8. Year, Month, Day, Day of week
* 날짜에 따른 변화를 알아보자.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(15, 5))
for col, ax in zip(['year', 'dow'], [ax[0], ax[1]]):
    sns.barplot(col, 'count', data=df_train, ax=ax)
    ax.set_title('AVG user by {}'.format(col))
f, ax = plt.subplots(2, 1, figsize=(15, 12))
for col, ax in zip(['month', 'day'], [ax[0], ax[1]]):
    sns.barplot(col, 'count', data=df_train, ax=ax)
    ax.set_title('AVG user by {}'.format(col))

* 2012년에 이용수가 증가했다.
* 일요일의 이용수가 상대적으로 낮다.
* 6~10월 에 이르며 이용수가 증가하며, 이후 감소한다.
* 날짜는 train set에는 19일까지, test set에는 20일부터 존재하므로 큰 의미가 없다.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(15, 5))
sns.barplot('year', 'registered', data=df_train, color=color[0], ax=ax[0], label='registered')
sns.barplot('year', 'casual', data=df_train, color=color[1], ax=ax[0], label='casual')
ax[0].set_title('AVG of user by year (seperated)')
ax[0].legend()

sns.barplot('dow', 'registered', data=df_train, color=color[0], ax=ax[1], label='registered')
sns.barplot('dow', 'casual', data=df_train, color=color[1], ax=ax[1], label='casual')
ax[1].set_title('AVG of user by dow (seperated)')
ax[1].legend(loc=1)

plt.figure(figsize=(15, 5))
sns.pointplot('hour', 'registered', data=df_train, color=color[0], label='registered')
sns.pointplot('hour', 'casual', data=df_train, color=color[1], label='casual')
plt.title('AVG of user by hour (seperated)')
plt.legend()

* 이용자별로 살펴보니, registered 연간 이용자 증가폭이 더 크다.
* registered 이용자는 주말 이용 감소를 보이는데 반해, casual 유저는 주말 이용이 증가한다.
* registered 이용자는 7-9시, 17-19시에 이용이 집중되어있다.
    * 출퇴근 시간으로, 직장인들의 비중이 높을것이라는 가설을 뒷받침해준다.
* casual은 오후시간대의 이용이 높다.